In [4]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import os
import time
import numpy as np
import random
from tqdm import notebook

### Stock list 

In [5]:
path = '/Users/ps712/OneDrive/ps/aibi/snsstock/'
path2='/Users/ps712/OneDrive/ps/aibi/snsstock/newsdata/'
list_name = 'stock.csv'
headers = {"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36"}
stock_list = pd.read_csv(os.path.join(path,list_name))
stock_list['종목코드'] = stock_list['종목코드'].apply(lambda x : str(x).zfill(6))
stock_list

,종목,종목코드
0,삼성전자,005930
1,카카오,035720
2,위메이드,112040
3,현대차,005380
4,SK하이닉스,000660
5,카카오뱅크,323410
6,한국조선해양,009540
7,ARIRANG 고배당주,161510
8,NAVER,035420
9,셀트리온,068270


In [6]:
#articles =[]

for code in notebook.tqdm(stock_list['종목코드'].values,"종목별 뉴스제목"):
    company_code = code
    result={}
    title_result=[]
    link_result =[]
    date_results = []
    source_results = []
    df = pd.DataFrame()
  
    url = 'https://finance.naver.com/item/news_news.nhn?code=' + str(company_code) + '&page=1'
    source_code = requests.get(url).text
    html = BeautifulSoup(source_code,"html.parser")

    lens = html.select('.pgRR')
    for len in lens: 
        a = len.find('a')['href']
    ab = int(a.split('=')[-1]) - 3

    for i in range(0,ab):
        page = i + 1   
        url = 'https://finance.naver.com/item/news_news.nhn?code=' + str(company_code) + '&page=' + str(page) 
        source_code = requests.get(url).text
        html = BeautifulSoup(source_code,"html.parser")
        titles = html.select('.title')
        
        for title in titles: 
            title = title.get_text() 
            title = re.sub('\n','',title)
            title_result.append(title)

        links = html.select('.title') 
        
        for link in links: 
            add = 'https://finance.naver.com' + link.find('a')['href']
            link_result.append(add)

        dates = html.select('.date')
        for date in dates:
            date = date.get_text()
            date_results.append(date)

        sources = html.select('.info')
        for source in sources:
            source = source.get_text()
            source_results.append(source)
            
        # for article in link_result:
        #     headers = {"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36"}
        #     response = requests.post(article, headers=headers)
        #     dom = BeautifulSoup(response.content.decode('euc-kr','replace'), "html.parser") 
        #     articles.append(dom.select_one("div#news_read").get_text())
        
    result= {"날짜" : date_results, "언론사" : source_results, "기사제목" : title_result, "링크" : link_result} 
    df=pd.DataFrame(result)
    df.to_csv(os.path.join(path2,company_code), encoding='utf8',index=True)

종목별 뉴스제목:   0%|          | 0/30 [00:00<?, ?it/s]

In [7]:
class news_crawler:

    def __init__(self):
        self.company_code_table = pd.read_csv('company_list.txt', dtype=str, sep='\t')


    def crawler(self, company_code, num_article):

        done_page_num=0

        # page = 1
        num_per_page=20 # naver serves 20 articles per page
        num_page,remainder=divmod(num_article,20)
        num_page+=1

        article_result=[]

        for page in range(done_page_num+1, done_page_num+num_page+1):
            try:
                url = 'https://finance.naver.com/item/news_news.nhn?code=' + str(company_code) + '&page=' + str(page)
                source_code = requests.get(url).text
                html = BeautifulSoup(source_code, "html.parser")


                # 뉴스 링크
                links = html.select('.title')

                link_result =[]
                if page == num_page:
                    links=links[:remainder]

                for link in links:
                    add = 'https://finance.naver.com' + link.find('a')['href']

                    link_result.append(add)

                print(f"{len(link_result)}개의 뉴스 크롤링..")
            except Exception:
                pass

            for article_url in link_result:
                try:
                    article_source_code = requests.get(article_url).text
                    article_html = BeautifulSoup(article_source_code, "lxml")
                    article_time = article_html.select('.tah')[0].get_text()

                    # 뉴스 내용
                    article_contents = article_html.select('.scr01')
                    article_contents=article_contents[0].get_text()
                    article_contents = re.sub('\n','',article_contents)
                    article_contents = re.sub('\t','',article_contents)

                    # cut extra text after Copyright mark
                    if "ⓒ" in article_contents:
                        article_contents=article_contents[:article_contents.index("ⓒ")]

                    # cut too long text to prevent CUDA OOM issue
                    if len(article_contents)>=1500:
                        article_contents=article_contents[:1500]

                    article_result.append([article_contents,article_time])

                    time.sleep(random.uniform(0.1,0.7))
                except Exception:
                    pass

            # print("다운 받고 있습니다------")

        return article_result


    def convert_company_to_code(self,company):

        # 종목코드 추출
        company_name = self.company_code_table['회사명']
        keys = [i for i in company_name]    #데이터프레임에서 리스트로 바꾸기

        company_code = self.company_code_table['종목코드']
        values = [j for j in company_code]

        dict_result = dict(zip(keys, values))  # 딕셔너리 형태로 회사이름과 종목코드 묶기

        pattern = '[a-zA-Z가-힣]+'

        if bool(re.match(pattern, company)) == True:
            company_code = dict_result.get(str(company))
            return company_code

        else:
            company_code = str(company)
            return company_code

    def crawl_news(self, company, max_num=5):
        print(f"{company} 종목 뉴스를 가져옵니다.")
        company_code=self.convert_company_to_code(company)

        if company_code:
                result=self.crawler(company_code, max_num)
                for i in range(len(result)):
                    result[i].append(company)
                return result

        else:
            print(f"{company} 종목이 존재하지 않습니다.")   
            return []